In [1]:
using InMemoryDatasets
import InMemoryDatasets as IMD
using BenchmarkTools
using TimerOutputs

# Data

In [2]:
@show dsl = Dataset(xid = [111,222,333,444,555], 
                    x1 = [1,2,1,4,5], 
                    x2 = [-1.2,-3,2.1,-3.5,2.8],
                    x3 = [Date("2019-10-03"), Date("2019-09-30"), Date("2019-10-04"), Date("2019-10-03"), Date("2019-10-03")],
                    x4 = ["abcd","efgh","ijkl","mnop","qrst"]);

@show dsr = Dataset(yid = [111,111,222,444,333],
                    y1 = [3,3,3,3,3],
                    y2 = [0,0, -3,1,2],
                    y3 = [Date("2019-10-01"),Date("2019-10-01"), Date("2019-09-30"), Date("2019-10-05"), Date("2019-10-05")],
                    y4 = ["abc","abcd","efg","mnop","qrst"]);

dsl_big = Dataset(rand(10^3, 5), :auto);
modify!(dsl_big,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsl_big,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));

dsr_big = Dataset(rand(10^3, 5), :auto);
modify!(dsr_big,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsr_big,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));
function x2y(str)
  replace(str,'x' => 'y')
end
rename!(x2y,dsr_big);

dsl_big2 = Dataset(rand(10^4, 5), :auto);
modify!(dsl_big2,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsl_big2,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));

dsr_big2 = Dataset(rand(10^4, 5), :auto);
modify!(dsr_big2,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsr_big2,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));
function x2y(str)
  replace(str,'x' => 'y')
end
rename!(x2y,dsr_big2);

dsl = Dataset(xid = [111, 222, 333, 444, 555], x1 = [1, 2, 1, 4, 5], x2 = [-1.2, -3, 2.1, -3.5, 2.8], x3 = [Date("2019-10-03"), Date("2019-09-30"), Date("2019-10-04"), Date("2019-10-03"), Date("2019-10-03")], x4 = ["abcd", "efgh", "ijkl", "mnop", "qrst"]) = 5×5 Dataset
 Row │ xid       x1        x2        x3          x4
     │ identity  identity  identity  identity    identity
     │ Int64?    Int64?    Float64?  Date?       String?
─────┼────────────────────────────────────────────────────
   1 │      111         1      -1.2  2019-10-03  abcd
   2 │      222         2      -3.0  2019-09-30  efgh
   3 │      333         1       2.1  2019-10-04  ijkl
   4 │      444         4      -3.5  2019-10-03  mnop
   5 │      555         5       2.8  2019-10-03  qrst
dsr = Dataset(yid = [111, 111, 222, 444, 333], y1 = [3, 3, 3, 3, 3], y2 = [0, 0, -3, 1, 2], y3 = [Date("2019-10-01"), Date("2019-10-01"), Date("2019-09-30"), Date("2019-10-05"), Date("2019-10-05")], y4 = ["abc", "abcd", "efg", "mnop",

# Version-4

In [139]:
function my_cartesianjoin_v4(dsl::AbstractDataset, dsr::AbstractDataset;
  on=nothing, threads::Bool=true, flag = ones(Bool,nrow(dsl) * nrow(dsr)))

  reset_timer!()

  dsr_cols = Symbol[]
  equalon_dsr_cols = Symbol[]
  conditions = Function[]

  for element in map(x -> x.second, on)
    if typeof(element) <: Pair
      push!(dsr_cols, element.first)
      push!(conditions, element.second)
    else
      push!(dsr_cols, element)
      push!(equalon_dsr_cols, element)
      push!(conditions, isequal)
    end
  end

  onleft = IMD.multiple_getindex(IMD.index(dsl), map(x -> x.first, on))
  onright = IMD.multiple_getindex(IMD.index(dsr), dsr_cols)

  equalon_dsr_cols = IMD.multiple_getindex(IMD.index(dsr), equalon_dsr_cols)
  right_cols = setdiff(1:length(IMD.index(dsr)), equalon_dsr_cols)

  oncols_left = onleft
  oncols_right = onright

  #println(oncols_left)
  #println(right_cols)

  # get flag, idx and ranges
  #println(flag)
  @timeit "compute flag vector"  cross_compare_vec(dsl, dsr, flag, conditions, onleft, onright, threads)
  #cross_compare(dsl,dsr,flag,conditions,onleft,onright,threads)
  #println(flag)

  l_len = nrow(dsl)
  r_len = nrow(dsr)

  @timeit "new ds" nds = generate_newds_v4(flag,dsl,dsr,l_len,r_len,right_cols)

  print_timer()
  flag,nds
end

function cross_compare_vec(dsl, dsr,
  flag, conditions, onleft, onright, threads)

  oncols_left = onleft
  oncols_right = onright

  l_len = nrow(dsl)
  r_len = nrow(dsr)

  for i in 1:length(conditions)  # Each conditions 每个条件
    fun = conditions[i]

    IMD.@_threadsfor threads for j in 1:l_len  # each row in dsl
      cur_index = (j-1)*l_len  
      _op_for_dsrcol(flag,fun,cur_index,IMD._columns(dsl)[oncols_left[i]][j],IMD._columns(dsr)[oncols_right[i]],r_len,
                     oncols_left,oncols_right)
      
    end
  end
  
end

function _op_for_dsrcol(flag,fun,cur_index,x,r_col,r_len,oncols_left,oncols_right)
  for k in 1:r_len
    flag[cur_index+k] == 0 && continue
    #println(IMD._columns(dsl)[oncols_left[i]][j],",",cur_index,",",IMD._columns(dsr)[oncols_right[i]][k])
    flag[cur_index+k] &= fun(x,r_col[k])         
  end
end


function generate_newds_v4(flag,dsl,dsr,l_len,r_len,right_cols)

  @time begin
      ## new left
      dsl_count = []  # 每一行对应几个右边
      for i in 1:l_len
          lo = 1+(i-1)*r_len
          hi = lo + r_len - 1
          push!(dsl_count,count(==(true),flag[lo:hi]))
      end
      
      new_ends = IMD.our_cumsum!(dsl_count)  # 累计和
      total_length = new_ends[end]
      
      res = []
      for j in 1:length(IMD.index(dsl))  # left 的每一列
        _res = IMD.allocatecol(IMD._columns(dsl)[j], total_length, addmissing=false)  # 
        
        x = IMD._columns(dsl)[j] # 这一步可以隔离
      
        # 左侧填充
        IMD.@_threadsfor true for i in 1:length(x)
          dsl_count[i] == 0 && continue
          i == 1 ? lo = 1 : lo = new_ends[i - 1] + 1
          hi = new_ends[i]
          IMD._fill_val_join!(_res, lo:hi, x[i])
        end
        
        push!(res, _res)
      end
  
      if dsl isa SubDataset
        newds = Dataset(res, copy(IMD.index(dsl)), copycols=false)
      else
        newds = Dataset(res, IMD.Index(copy(IMD.index(dsl).lookup), copy(IMD.index(dsl).names), copy(IMD.index(dsl).format)), copycols=false)
      end
  end

  
  @time begin
  ## new right
  #println("Cerating right")

  dsr_idx = []
  for i in 1:l_len
      lo = 1+(i-1)*r_len
      hi = lo + r_len - 1
      append!(dsr_idx,findall(isone,flag[lo:hi]))
  end 

  #println(dsr_idx)

  #println("Nesting")
  for j in 1:length(right_cols)
    _res = IMD.allocatecol(IMD._columns(dsr)[right_cols[j]], total_length, addmissing=false)  # 空的dsr

    IMD.@_threadsfor true for i in 1:length(l_len)
      #print(j)
      length(dsr_idx[i]) == 0 && continue
      for i in 1:length(dsr_idx)
        _res[i] = IMD._columns(dsr)[right_cols[j]][dsr_idx[i]]
      end
    end

    #println(_res)
    push!(IMD._columns(newds), _res)

    new_var_name = IMD.make_unique([IMD._names(dsl); IMD._names(dsr)[right_cols[j]]], makeunique=true)[end]
    push!(IMD.index(newds), new_var_name)
    setformat!(newds, IMD.index(newds)[new_var_name], getformat(dsr, IMD._names(dsr)[right_cols[j]]))
  end
end
  newds

  
end

generate_newds_v4 (generic function with 1 method)

# Version-5
V4 的函数隔离版本

In [332]:
function my_cartesianjoin_v5(dsl::AbstractDataset, dsr::AbstractDataset;
  on=nothing, threads::Bool=true, flag = ones(Bool,nrow(dsl) * nrow(dsr)))

  reset_timer!()

  dsr_cols = Symbol[]
  equalon_dsr_cols = Symbol[]
  conditions = Function[]

  for element in map(x -> x.second, on)
    if typeof(element) <: Pair
      push!(dsr_cols, element.first)
      push!(conditions, element.second)
    else
      push!(dsr_cols, element)
      push!(equalon_dsr_cols, element)
      push!(conditions, isequal)
    end
  end

  onleft = IMD.multiple_getindex(IMD.index(dsl), map(x -> x.first, on))
  onright = IMD.multiple_getindex(IMD.index(dsr), dsr_cols)

  equalon_dsr_cols = IMD.multiple_getindex(IMD.index(dsr), equalon_dsr_cols)
  right_cols = setdiff(1:length(IMD.index(dsr)), equalon_dsr_cols)

  oncols_left = onleft
  oncols_right = onright

  #println(oncols_left)
  #println(right_cols)

  # get flag, idx and ranges
  #println(flag)
  @timeit "compute flag vector"  cross_compare_vec(dsl, dsr, flag, conditions, onleft, onright, threads)
  #cross_compare(dsl,dsr,flag,conditions,onleft,onright,threads)
  #println(flag)

  l_len = nrow(dsl)
  r_len = nrow(dsr)

  @timeit "new ds" nds = generate_newds_v5(flag,dsl,dsr,l_len,r_len,right_cols)

  print_timer()
  flag,nds
end

function cross_compare_vec(dsl, dsr,
  flag, conditions, onleft, onright, threads)

  oncols_left = onleft
  oncols_right = onright

  l_len = nrow(dsl)
  r_len = nrow(dsr)

  for i in 1:length(conditions)  # Each conditions 每个条件
    fun = conditions[i]

    IMD.@_threadsfor threads for j in 1:l_len  # each row in dsl
      cur_index = (j-1)*l_len  
      _op_for_dsrcol(flag,fun,cur_index,IMD._columns(dsl)[oncols_left[i]][j],IMD._columns(dsr)[oncols_right[i]],r_len,
                     oncols_left,oncols_right)
      
    end
  end
  
end

function _op_for_dsrcol(flag,fun,cur_index,x,r_col,r_len,oncols_left,oncols_right)
  for k in 1:r_len
    flag[cur_index+k] == 0 && continue
    #println(IMD._columns(dsl)[oncols_left[i]][j],",",cur_index,",",IMD._columns(dsr)[oncols_right[i]][k])
    flag[cur_index+k] &= fun(x,r_col[k])         
  end
end

function generate_newds_v5(flag,dsl,dsr,l_len,r_len,right_cols,threads=true)
  T = Int32

  ## new left
  ### step-1
  dsl_count = Vector{T}(undef, nrow(dsl))  # left每一行对应几个右边
  
  find_count_for_left(flag, dsl_count,l_len,r_len)
  
  new_ends = cumsum(dsl_count)  # 累计和
  total_length = new_ends[end]
  
  ### step-2
  res = []  # 固定尺寸不用push？
  for j in 1:length(IMD.index(dsl))  # left 的每一列
    _res = IMD.allocatecol(IMD._columns(dsl)[j], total_length, addmissing=false)  # 
    
    fill_left_res(_res, IMD._columns(dsl)[j], dsl_count, new_ends, threads)

    push!(res, _res)
    
  end
  
### step-3
  if dsl isa SubDataset
    newds = Dataset(res, copy(IMD.index(dsl)), copycols=false)
  else
   newds = Dataset(res, IMD.Index(copy(IMD.index(dsl).lookup), copy(IMD.index(dsl).names), copy(IMD.index(dsl).format)), copycols=false)
  end

#println(newds)
  

  ## new right
  #println("Cerating right")
  @timeit "findall" begin
  ### step-4
   dsr_idx = findall(isone,flag)  # 全局索引，要去做出局部索引
  for i in 1:l_len
      dsl_count[i] == 0 && continue
      
      i == 1 ? lo = 1 : lo = new_ends[i - 1] + 1
      hi = new_ends[i]

      dsr_idx[lo:hi] .-= (i-1)*r_len
      #for k in lo:hi
      #    all[k] -= (i-1)*r_len
      #end
  end 
end

@timeit "newds left" begin
  ### step-5
  #println("Nesting")
  for j in 1:length(right_cols)   # right 的每一列
    _res = IMD.allocatecol(IMD._columns(dsr)[right_cols[j]], total_length, addmissing=false)  # 空的dsr

    fill_right_res(_res,IMD._columns(dsr)[right_cols[j]],dsr_idx,threads)

    #println(_res)
    push!(IMD._columns(newds), _res)
    new_var_name = IMD.make_unique([IMD._names(dsl); IMD._names(dsr)[right_cols[j]]], makeunique=true)[end]
    push!(IMD.index(newds), new_var_name)
    setformat!(newds, IMD.index(newds)[new_var_name], getformat(dsr, IMD._names(dsr)[right_cols[j]]))
  end
end

  newds
end

function find_count_for_left(flag, dsl_count,l_len,r_len)
  for i in 1:l_len
      lo = 1+(i-1)*r_len
      hi = lo + r_len - 1
      #push!(dsl_count,count(==(true),flag[lo:hi]))
      @inbounds dsl_count[i] = count(view(flag,lo:hi))
  end
end

function fill_left_res(_res, l_col, dsl_count, new_ends, threads)
  # x = IMD._columns(dsl)[j] # 这一步可以隔离
  # 左侧填充
  IMD.@_threadsfor threads for i in 1:length(l_col)
    dsl_count[i] == 0 && continue
    i == 1 ? lo = 1 : lo = new_ends[i - 1] + 1
    hi = new_ends[i]
    IMD._fill_val_join!(_res, lo:hi, l_col[i])
  end
end

function fill_right_res(_res, r_col, dsr_idx, threads)
  IMD.@_threadsfor threads for i in 1:length(dsr_idx) # 1:length(l_len)  # left 的每一列
    _res[i] = r_col[dsr_idx[i]]
  end
end

fill_right_res (generic function with 1 method)

# Draft

In [157]:
dsr

5×5 Dataset
 Row │ yid       y1        y2        y3          y4       
     │ identity  identity  identity  identity    identity 
     │ Int64?    Int64?    Int64?    Date?       String?  
─────┼────────────────────────────────────────────────────
   1 │      111         3         0  2019-10-01  abc
   2 │      111         3         0  2019-10-01  abcd
   3 │      222         3        -3  2019-09-30  efg
   4 │      444         3         1  2019-10-05  mnop
   5 │      333         3         2  2019-10-05  qrst

In [4]:
function fun1(x,y) 
  x <= y
end

fun1 (generic function with 1 method)

In [324]:
flag,nds = my_cartesianjoin_v5(dsl,dsr,
  on = [:xid=>:yid, :x2=>:y2=>fun1]); # :x1=>:y1=>isless, 
nds

  0.000012 seconds (16 allocations: 1.156 KiB)
  0.000563 seconds (309 allocations: 25.453 KiB)
 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:           1.81ms /  97.9%           64.4KiB /  95.2%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 new ds                     1   1.30ms   73.4%  1.30ms   52.8KiB   86.1%  52.8KiB
   findall                  1   7.76μs    0.4%  7.76μs      224B    0.4%     224B
 compute flag vector        1    471μs   26.6%   471μs   8.50KiB   13.9%  8.50KiB
 ────────────────────────────────────────────────────────────────────────────────


4×9 Dataset
 Row │ xid       x1        x2        x3          x4        y1        y2        ⋯
     │ identity  identity  identity  identity    identity  identity  identity  ⋯
     │ Int64?    Int64?    Float64?  Date?       String?   Int64?    Int64?    ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │      111         1      -1.2  2019-10-03  abcd             3         0  ⋯
   2 │      111         1      -1.2  2019-10-03  abcd             3         0
   3 │      222         2      -3.0  2019-09-30  efgh             3        -3
   4 │      444         4      -3.5  2019-10-03  mnop             3         1
                                                               2 columns omitted

In [334]:
flag,nds = my_cartesianjoin_v5(dsl_big,dsr_big,
  on = [:x1 => :y1, :x2 => :y2 => fun1],threads=true);

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:           5.15ms /  98.9%            793KiB /  99.6%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 new ds                     1   3.46ms   67.8%  3.46ms    704KiB   89.1%   704KiB
   newds left               1    780μs   15.3%   780μs    198KiB   25.1%   198KiB
   findall                  1    696μs   13.7%   696μs    255KiB   32.3%   255KiB
 compute flag vector        1   1.64ms   32.2%  1.64ms   86.3KiB   10.9%  86.3KiB
 ────────────────────────────────────────────────────────────────────────────────


In [336]:
flag,nds = my_cartesianjoin_v5(dsl_big2,dsr_big2,
  on = [:x1 => :y1, :x2 => :y2 => fun1],threads=true);

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            247ms / 100.0%           60.3MiB / 100.0%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 new ds                     1    166ms   67.2%   166ms   59.4MiB   98.5%  59.4MiB
   findall                  1   37.7ms   15.3%  37.7ms   20.3MiB   33.7%  20.3MiB
   newds left               1   9.25ms    3.7%  9.25ms   17.3MiB   28.7%  17.3MiB
 compute flag vector        1   81.2ms   32.8%  81.2ms    930KiB    1.5%   930KiB
 ────────────────────────────────────────────────────────────────────────────────


In [303]:
  l_len = nrow(dsl_big)
  r_len = nrow(dsr_big)

1000

In [269]:
  l_len = nrow(dsl)
  r_len = nrow(dsr)

5

## step4-findall优化

In [320]:
@time begin
  all = findall(isone,flag);
end

  0.052868 seconds (8 allocations: 15.770 MiB)


503912-element Vector{Int64}:
      136
      300
      424
     1040
     1288
     1765
     2271
     2719
     3164
     3249
        ⋮
 99989583
 99989724
 99989757
 99989999
 99991146
 99991195
 99993133
 99993603
 99995943

In [310]:
@time begin
  all = Vector{Int32}(undef,total_length)
  ct = 1
  for i in 1:length(flag)
    flag[i] == 0 && continue
    all[ct] = i
    ct += 1
  end
end

  0.124739 seconds (3.00 M allocations: 61.105 MiB)


## step1优化

In [304]:
T = Int32
@time begin
  ## new left
  dsl_count = Vector{T}(undef, nrow(dsl_big))  # left每一行对应几个右边

  """
  for i in 1:l_len
    lo = 1+(i-1)*r_len
    hi = lo + r_len - 1
    #push!(dsl_count,count(==(true),flag[lo:hi]))
    @inbounds dsl_count[i] = count(view(flag,lo:hi))
  end
  """
  
  find_count_for_left(flag, dsl_count,l_len,r_len)
end

  new_ends = cumsum(dsl_count)  # 累计和
  total_length = new_ends[end]



  0.001153 seconds (3 allocations: 4.094 KiB)


4898

## step2优化

In [ ]:
@time IMD._columns(dsl)[1]

## step-4优化

In [190]:


  #dsr_idx = []
@time begin
  dsr_idx = findall(isone,flag) # Vector{T}(undef,total_length)
  #all = findall(isone,flag)


  for i in 1:l_len

    """
      lo = 1+(i-1)*r_len
      hi = lo + r_len - 1
      # append!(dsr_idx,findall(isone,flag[lo:hi]))
      #append!(dsr_idx,findall(isone,view(flag,(i-1)*r_len + 1:i*r_len)))
      """

      dsl_count[i] == 0 && continue

      """
      lo = 1+(i-1)*r_len
      hi = lo + r_len - 1
      pat_res = findall(isone,view(flag,lo:hi))
      """

      i == 1 ? lo = 1 : lo = new_ends[i - 1] + 1
      hi = new_ends[i]
      

      #println(lo:hi)
      #println(all[lo:hi] .- (i-1)*r_len)
      #dsr_idx[lo:hi] = findall(isone,view(flag,(i-1)*r_len + 1:i*r_len))
      #dsr_idx[lo:hi] = all[lo:hi] .- (i-1)*r_len #broadcast(-,all[lo:hi],(i-1)*r_len)
      
      
      dsr_idx[lo:hi] .-= (i-1)*r_len

      # for k in lo:hi
      #    all[k] -= (i-1)*r_len
      # end
      
    end 

end

  0.001566 seconds (10.69 k allocations: 494.156 KiB)


In [163]:
@time findall(isone,view(flag,1:5));
@time findall(isone,flag);

  0.000018 seconds (6 allocations: 320 bytes)
  0.000021 seconds (5 allocations: 224 bytes)


In [81]:
@time  count(view(flag,1:5))

  0.000015 seconds (1 allocation: 48 bytes)


2

In [173]:
@time all[1:2] .-= 1

  0.000024 seconds (4 allocations: 192 bytes)


2-element view(::Vector{Int64}, 1:2) with eltype Int64:
 -5
 -4

In [172]:
@time begin
  for i in 1:2
    all[i] -= 1
  end
end

  0.000011 seconds


In [92]:
@time  begin
for i in 1:length(view(flag,1:5))
  view(flag,1:5)[i] == 0 && continue
  append!(di,1)
end
end

  0.000031 seconds (12 allocations: 480 bytes)


In [340]:
a = [1,2,3]
eachindex(a)

Base.OneTo(3)

In [342]:
for i in eachindex(a)
  print(i)
end


123

In [215]:
flag

25-element Vector{Bool}:
 1
 1
 0
 0
 0
 0
 0
 1
 0
 0
 ⋮
 0
 0
 1
 0
 0
 0
 0
 0
 0